In [1]:
spark.version

'2.3.0'

In [31]:
import numpy as np
import murmurhash as mmh

In [5]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType,StructField, StringType
from collections import defaultdict

In [34]:
sc.stop()

In [35]:
conf = SparkConf().setAppName('testGroupBy').setMaster('local[*]')
sc = SparkContext(conf=conf)

In [36]:
sc.textFile('test.txt')

test.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [37]:
file = sc.textFile('data_test1.csv').map(lambda line: line.split(";")) \
    .filter(lambda line: len(line)>1)\
    .filter(lambda line:not line[0]=='id')\
    .map(lambda line: line + [hash(line[1])%10])

In [ ]:
file.collect()

In [71]:

def groupByHash(file, by, agg, agg_on,early_agg = True):
    def hashPartition(iterator):
        buckets = defaultdict(list)
        if early_agg:
            for obj in iterator:
                if obj[by] in buckets:
                    if agg == 'count':
                        buckets[obj[by]] += 1 
                    if agg == 'sum':
                        buckets[obj[by]] += float(obj[agg_on])
                    if agg =='max':
                        buckets[obj[by]] = max(buckets[obj[by]],float(obj[agg_on]))
                else:
                    if agg == 'count':
                        buckets[obj[by]] = 1 
                    if agg == 'sum':
                        buckets[obj[by]] = float(obj[agg_on])
                    if agg =='max':
                        buckets[obj[by]] = float(obj[agg_on])


        
        if not early_agg:
            for obj in iterator:
                buckets[obj[by]] += [obj[agg_on]]
            for k,v in buckets.items():
                if agg == 'count':
                    buckets[k] = len(v)
                if agg == 'sum':
                    buckets[k] = np.sum(np.array(v,dtype=np.float32))
                if agg =='max':
                    buckets[k] = np.max(np.array(v,dtype=np.float32))        
        yield buckets


    def merge(m1, m2):
        for k, v in m2.items():
            m1[k] += v
        return m1

    return file.mapPartitions(hashPartition).reduce(merge)

    

In [72]:
groupByHash(file,1,'max',3,early_agg=True)

defaultdict(list,
            {'Student': 132.7,
             'Admin': 128.8,
             'Teacher': 124.9,
             'Lecturer': 135.5})